In [ ]:
import pygame
import sys
import random
import math

from pygame.locals import *

In [ ]:
# setup game
pygame.init()
mainClock = pygame.time.Clock()


# create window
WINDOWWIDTH = 600
WINDOWHEIGHT = 600

screen = pygame.display.set_mode((WINDOWWIDTH,WINDOWWIDTH), 0, 32)
screen.fill((0,194,249))
pygame.display.set_caption('Catan')


# location of settlements
settlementLoc = []

# resources
BRICK = (255, 0, 0)
ORE = (190, 190, 190)
WHEAT = (255, 215, 0)
SHEEP = (124, 252, 0)
WOOD = (0, 100, 0)
DESSERT = (139, 69, 19)

resources = {BRICK: 'Brick', ORE: 'Ore', WHEAT : 'Wheat',
            SHEEP : 'Sheep', WOOD:'Wood', DESSERT:'Dessert'}

# players
P1 = (0, 0, 255)
P2 = (255, 20, 147)
P3 = (255, 165, 0)
P4 = (70, 130, 180)

# hex locations
hexTiles = [(0,0), (0,1), (0,-1), (-1,0.5), (-1,-0.5), (1,0.5), (1,-0.5), (2,0), (-2,0), (0,2),
                (0,-2), (1,1.5), (1,-1.5), (-1,1.5), (-1, -1.5), (2,1), (2,-1), (-2,1), (-2,-1)]

hexProbabilities = [5, 2, 6, 3, 8, 10, 9, 12, 11, 4, 8, 10, 9, 4, 5, 6, 3, 11]

random.shuffle(hexProbabilities)

hexResources = [BRICK, BRICK, BRICK, ORE, ORE, ORE, WHEAT, WHEAT, WHEAT, WHEAT, SHEEP, 
                SHEEP, SHEEP, SHEEP, WOOD, WOOD, WOOD, WOOD]

zipHex = list(zip(hexResources, hexProbabilities))

zipHex.append((DESSERT, 7))


random.shuffle(zipHex)

# Blob

In [ ]:
[tile for tile in zip(hexTiles, zipHex)]

In [ ]:
# Board Class
class Board:
    hexes = []
    settlements = []
    instructions = ""
    
    def __init__(self, size):
        self.cX = screen.get_rect().centerx
        self.cY = screen.get_rect().centery
        
        self.hexHeight = 0.866 * size
        
        # the hexs
        for tile in zip(hexTiles, zipHex):
            self.hexes.append(Hex(tile[0], size, tile[1][0], tile[1][1], self))
        
        # set up the settlement locations
        self.settlements = settlementLoc # may not to do anymore

            
    def draw(self):
        # draw the HEXes
        for h in self.hexes:
            h.draw()

        # draw the settlement locations
        for item in self.settlements:
            item.draw()

        # message to user
        basicFont = pygame.font.SysFont(None, 24)
        text = basicFont.render("Select your first village", True, (255,255,255), (0,0,0))
        textRect = text.get_rect()
        textRect.left = 5
        textRect.top = 560
        screen.blit(text, textRect)
        
""" Create a settlement class"""
class Settlement:
    def __init__(self, location, tileOwner):
        self.location = location
        self.locationx = location[0]
        self.locationy = location[1]
        self.onTiles = [tileOwner]
        self.neighbours = [] # do I need?
        self.city = False
        self.owner = -1
        
    def draw(self):
        pygame.draw.rect(screen, (0,0,0), (self.locationx, self.locationy, 20, 20),2)

        
# Hex Class
class Hex:

    def __init__(self, centre, size, resource, probability, board):
        self.centre = centre
        self.hexX = board.cX +(1.5*centre[0]*size)
        self.hexY = board.cY+(2*centre[1]*board.hexHeight)
        self.hexSize = size
        self.resource = resource
        self.value = probability	# chance of getting this resource
        self.hasRobber = False
        
        # calculate each 6 points so we only do it once
        self.points = []
        self.points.append((self.hexX - self.hexSize, self.hexY))
        self.points.append((self.hexX - (self.hexSize/2), self.hexY - (0.866*self.hexSize)))
        self.points.append((self.hexX + (self.hexSize/2), self.hexY - (0.866*self.hexSize)))
        self.points.append((self.hexX + self.hexSize, self.hexY))
        self.points.append((self.hexX + (self.hexSize/2), self.hexY + (0.866*self.hexSize)))
        self.points.append((self.hexX - (self.hexSize/2), self.hexY + (0.866*self.hexSize)))
        
        # add the points of all settlements to the settlement lists
        global settlementLoc
        newSets = [] # store all the locations of the new settlements from this til
        newSets += [((10*math.floor(X / 10))-5, (10*math.floor(Y / 10))-5) for (X,Y) in self.points]
        
        for ns in newSets:	# these are the ones for the current hex
            for s in settlementLoc:	# these are the ones that have been established
                if s.location == ns:		# so if currently is an established settlement
                    newSets.remove(ns)	# remove it from the set
                    s.onTiles.append(self)	# add to that settlement's tile owneres
                    break
                    
        for ns in newSets:	# these are the remainders
            settlementLoc.append(Settlement(ns, self))
        
    def draw(self): 
        # draw the hex
        pygame.draw.polygon(screen, self.resource, self.points, 0)
        
        # draw the text
        if self.value > 0:
            font = pygame.font.Font(None, 36)
            text = font.render(str(self.value), 1, (0,0,0))
            textpos = text.get_rect()
            textpos.left = self.hexX - 5
            textpos.top = self.hexY - 10
            screen.blit(text, textpos)



In [ ]:
""" Set up the information for the settlements """
settlers = Board(60)

# set up the locations of each house
roads = []
# run the game loop
while True:
    # check for the QUIT event
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == MOUSEBUTTONDOWN:
            click = pygame.mouse.get_pos()
            print (click)
            x = (10*math.floor(click[0] / 10))-5
            y = (10*math.floor(click[1] / 10))-5
            for sett in settlers.settlements:
                if sett.location == (x,y):
                    for tile in sett.onTiles:
                        print (str(resources[tile.resource]) + " is " + str(tile.value))


    ## draw the board        
    settlers.draw()

    # draw the window onto the screen
    pygame.display.update()
    mainClock.tick(40)


In [ ]:
settlers.settlements